# Utilities to test DynamoDb and S3 database for TEMCA Optical images

In [1]:
import boto3

# keep the following consistent per run
specimen_id = 'donaldtrump'
media_id = 'test1'
s3bucket = 'temcatester'
region = 'us-west-2'

In [2]:
dynamodb = boto3.resource('dynamodb', region_name=region)

In [14]:
# Create the TAO table.  Just do this once.
table = dynamodb.create_table(
    TableName='TAO',
    KeySchema=[
        {
            'AttributeName': 'specimenId_mediaId',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'grid',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'specimenId_mediaId',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'grid',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)

print("Table status:", table.table_status)

('Table status:', u'CREATING')


In [3]:
table = dynamodb.Table('TAO')
print("Table status:", table.table_status)

('Table status:', u'ACTIVE')


In [5]:
# Create the S3 bucket
s3 = boto3.resource('s3')
s3.create_bucket(Bucket=s3bucket, CreateBucketConfiguration={'LocationConstraint': region})

ClientError: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

In [9]:
# push images and TAOs into the DB
import os
from subprocess import Popen, PIPE

rootdir = r"\\AIBSDATA2\nc-em\adamb\atumDevelopment\tapeRuns\20160721\20160721_sectionImages"
count = 0

bucket = s3.Bucket(s3bucket)
table = dynamodb.Table('TAO')

for root, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith(".jpeg"):
            fpath = os.path.join(root, file) 
            p = Popen(['zbarimg.exe', '--raw', fpath], stdout=PIPE, stderr=PIPE, stdin=PIPE)
            barcode = p.stdout.read() 
            
            print count, barcode, fpath
            count = count + 1
            print len(barcode)
            if (len(barcode) != 7):
                print ('barcode decode failed for: ' + fpath)
            else:
                try:
                    tao = {
                        'specimen_id' : specimen_id,
                        'media': 'tape',
                        'media_id': media_id,
                        'grids': {
                            barcode : {
                                rois: []
                            }
                        }
                    }
                    
                    item = {'specimenId_mediaId': specimen_id + "_" + media_id,
                            'grid' : barcode,
                            'tao': tao,
                            'bucket': s3bucket,
                            'aperture_im': file}
                    
                    response = table.putitem(item)
                    print response
                    response = s3.Bucket(s3bucket).upload_file(file, fpath)
                    print response
                    print ('ok')
                except:
                    print('upload failed: ' + file)
            if (count > 3):
                break



0 000051
\\AIBSDATA2\nc-em\adamb\atumDevelopment\tapeRuns\20160721\20160721_sectionImages\160721_111222ximeatest.jpeg
7
upload failed: 160721_111222ximeatest.jpeg
1 000051
\\AIBSDATA2\nc-em\adamb\atumDevelopment\tapeRuns\20160721\20160721_sectionImages\160721_111223ximeatest.jpeg
7
upload failed: 160721_111223ximeatest.jpeg
2  \\AIBSDATA2\nc-em\adamb\atumDevelopment\tapeRuns\20160721\20160721_sectionImages\160721_111224ximeatest.jpeg
0
barcode decode failed for: \\AIBSDATA2\nc-em\adamb\atumDevelopment\tapeRuns\20160721\20160721_sectionImages\160721_111224ximeatest.jpeg
3  \\AIBSDATA2\nc-em\adamb\atumDevelopment\tapeRuns\20160721\20160721_sectionImages\160721_111225ximeatest.jpeg
0
barcode decode failed for: \\AIBSDATA2\nc-em\adamb\atumDevelopment\tapeRuns\20160721\20160721_sectionImages\160721_111225ximeatest.jpeg
